## BASICS OF LANGGRAPH

This LangGraph project demonstrates a simple, sequential workflow for calculating financial figures. The core components are a `TypedDict` state schema (`Person_State`) defining shared variables like `principle`, `rate`, and `time`. The workflow consists of two nodes: `calc_SI` calculates the Simple Interest (SI), and `account_bal` uses that result to calculate the final `balance`. The graph runs sequentially from `START` to `SI_Calculator`, then to `Balance_calculator`, and finally to `END`. It successfully compiles and executes, showing the final state containing the inputs, calculated SI, and the final balance. The graph structure is also visualized using ASCII art.

### Import necessary components for building the graph.

In [39]:
# Import necessary components for building the graph.
# TypedDict is used to define the schema (structure and types) of the graph's state.
from typing_extensions import TypedDict, Optional

# StateGraph is the main class for building state-based workflows.
# START and END are special nodes defining the entry and exit points of the graph.
from langgraph.graph import StateGraph, END, START

### Define the Graph State Schema (Person_State).

In [40]:
# This dictionary holds all information shared and updated by the nodes in the graph.
class Person_State(TypedDict):
    # Input parameters for the Simple Interest calculation:
    principle : int
    rate : float
    time : int
    # Output fields. Initially set to 'None' as they will be calculated by the nodes.
    SI : None # Simple Interest (will be a float after calculation) # Alternative code SI : Optional[float]
    balance : None # Account Balance (will be a float after calculation)

### Define Node Function 1: Simple Interest Calculator.

In [41]:

# All node functions receive the current graph state (Person_State) as input.
def calc_SI(state:Person_State) -> Person_State:
    # Calculation: SI = P * R * T / 100
    si = state['principle']*state['rate']*state['time']/100
    # The function returns a dictionary of state updates. 
    # LangGraph automatically merges this {"SI": si} into the main 'state' object.
    return {"SI":si}

### Define Node Function 2: Account Balance Calculator.

In [42]:

# This node runs after 'calc_SI' and can use the calculated 'SI' value from the state.
def account_bal(state: Person_State) -> Person_State:
    # Calculation: Balance = Principle + Simple Interest
    bal = state['SI'] + state['principle']
    # Update the state with the calculated balance.
    return {'balance':bal}

### Build the Graph Structure

In [43]:

# Initialize the graph builder using the defined state schema (Person_State).
builder = StateGraph(Person_State)

# Add the functions as executable nodes to the graph.
builder.add_node('SI_Calculator',calc_SI)
builder.add_node('Balance_calculator',account_bal)

# Define the workflow (edges/transitions):
# 1. Start execution at 'SI_Calculator' using the input data.
builder.add_edge(START,'SI_Calculator')
# 2. After 'SI_Calculator' runs, transition to 'Balance_calculator'.
builder.add_edge('SI_Calculator','Balance_calculator')
# 3. After 'Balance_calculator' runs, terminate the graph execution.
builder.add_edge('Balance_calculator',END)

# Compile the graph into a runnable object (ready for execution).
graph = builder.compile()


### Run the Graph (Invocation)

In [44]:

# The 'invoke' method runs the graph with the initial state provided.
# The input dictionary is merged into the Person_State and used by the nodes in sequence.
# The final result is the complete state dictionary after all nodes have executed.
graph.invoke({'principle':56000,"rate":5.5,'time':2})

{'principle': 56000, 'rate': 5.5, 'time': 2, 'SI': 6160.0, 'balance': 62160.0}

### Visualize the Graph

In [45]:
# Install dependency for ASCII visualization
# This package is required by LangGraph to correctly format the graph into ASCII art.
# !pip install grandalf

In [46]:
# get_graph() retrieves the structure of the compiled graph.
# draw_ascii() outputs a text-based, tree-like representation of the node flow.
print(graph.get_graph().draw_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
  +---------------+    
  | SI_Calculator |    
  +---------------+    
           *           
           *           
           *           
+--------------------+ 
| Balance_calculator | 
+--------------------+ 
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      
